In [ ]:
#Import necessary packges
import numpy as np
import matplotlib.pyplot as plt

## Poisson autoregression

In [ ]:
###CREATE NECESSARY FUNCTIONS

# Define excitation function psi using a modified log-sum-exp function with clipping to avoid overflow
def psi(x):
    return np.clip(np.log(1 + np.exp(x)), None, 40)

# Function to sample from Poisson distribution with intensity psi(lambda) * timestep
def poisson_randomness(lmbd, timestep):
    E = np.random.exponential(scale=1.0, size=10)  # Generate 10 random samples from an exponential distribution
    i = 0
    S = 0  
    while S <= psi(lmbd) * timestep:  
        i += 1
        S += E[i]  
    return i -1 

# Function to compute the next value of lambda (lmbd) based on its previous value and the previous event count X
def lmbd_next(lmbd_prev, X_prev, timestep=1, alpha=0.9, beta=0.5, mu=1):
    # Compute the next lambda value using the autoregressive formula
    lmbd_next = mu * (1 - np.exp(-beta * timestep)) + np.exp(-beta * timestep) * lmbd_prev + alpha * np.exp(-beta * timestep) * X_prev
    return lmbd_next


In [ ]:
###SIMULATION FUNCTION

# Function to simulate the Poisson autoregressive model over time
def simulate(T, timestep=0.01, alpha=2, beta=5, mu=2):
    """
    Simulate a Poisson autoregressive model with the following parameters:
        T: Final time
        timestep: Time increment for each step
        alpha: Weight of the previous count in the lambda update
        beta: Decay rate of lambda
        mu: Baseline intensity for lambda
    """
    N = np.zeros(T)  # Array to store the cumulative counts
    lmbd = np.zeros(T)  # Array to store lambda values
    X = np.zeros(T)  # Array to store the event counts at each time step
    
    # Initialize the first values
    lmbd[0] = mu  # Start with the baseline intensity
    X[0] = 0  # Set inital count to zero
    N[0] = X[0]  # Initialize cumulative count

    # Evolve the process until final time T
    for i in range(1, T): 
        lmbd[i] = lmbd_next(lmbd[i-1], X[i-1], timestep, alpha, beta, mu)  # Update lambda
        X[i] = poisson_randomness(lmbd[i], timestep)  # Sample new event count based on updated lambda
        N[i] = N[i-1] + X[i]  # Update cumulative count

    return N, lmbd, X  # Return the cumulative counts, lambda values, and individual event counts


In [ ]:
#Function to plot the Poisson autoregressive model for two different alpha values.
def plot_ar(T, alpha_values):
    """
    Parameters:
        T (int): Total number of time steps.
        alpha_values (list): List of two alpha values to compare.
    """
    # Adjust the font sizes for various elements in the plot
    plt.rcParams.update({
        'font.size': 16,         # Default font size for all elements
        'axes.titlesize': 18,    # Font size for subplot titles
        'axes.labelsize': 16,    # Font size for x and y labels
        'xtick.labelsize': 14,   # Font size for x tick labels
        'ytick.labelsize': 14,   # Font size for y tick labels
        'legend.fontsize': 14    # Font size for legend labels
    })

    # Simulate the model for two different alpha values
    N1, lmbd1, X1 = simulate(T, alpha=alpha_values[0])  # First simulation with alpha_1
    N2, lmbd2, X2 = simulate(T, alpha=alpha_values[1])  # Second simulation with alpha_2

    # Create a figure with 2x2 subplots, specifying the overall figure size
    fig, axes = plt.subplots(2, 2, figsize=(20, 8))

    # First subplot: Plot the event counts (X1) over time for the first alpha value
    axes[0, 0].plot(X1, 'o', color='red', label='X1')
    axes[0, 0].set_title('X1')  # Title of the subplot
    axes[0, 0].set_xlabel('Time')  # Label for the x-axis
    axes[0, 0].set_ylabel('Jump')  # Label for the y-axis
    axes[0, 0].legend()  # Show legend
    axes[0, 0].grid(True)  # Enable grid for better readability

    # Second subplot: Plot lambda (λ1) and its transformed value ψ(λ1) for the first alpha value
    axes[0, 1].plot(lmbd1, '^', color='blue', label='λ1')  # Plot lambda values using triangles
    axes[0, 1].plot(psi(lmbd1), '*', color='orange', label=r'$\psi(\lambda_1)$')  # Plot psi(lambda) values with stars
    axes[0, 1].set_title('λ1 and ψ(λ1)')  # Title of the subplot
    axes[0, 1].set_xlabel('Time')  # Label for the x-axis
    axes[0, 1].set_ylabel('Intensity')  # Label for the y-axis
    axes[0, 1].legend()  # Show legend
    axes[0, 1].grid(True)  # Enable grid for better readability

    # Third subplot: Plot the event counts (X2) over time for the second alpha value
    axes[1, 0].plot(X2, 'o', color='lightcoral', label='X2')  # Plot X2 with circular markers
    axes[1, 0].set_title('X2')  # Title of the subplot
    axes[1, 0].set_xlabel('Time')  # Label for the x-axis
    axes[1, 0].set_ylabel('Jump')  # Label for the y-axis
    axes[1, 0].legend()  # Show legend
    axes[1, 0].grid(True)  # Enable grid for better readability

    # Fourth subplot: Plot lambda (λ2) and its transformed value ψ(λ2) for the second alpha value
    axes[1, 1].plot(lmbd2, '^', color='lightblue', label='λ2')  # Plot lambda values with light blue triangles
    axes[1, 1].plot(psi(lmbd2), '*', color='peachpuff', label=r'$\psi(\lambda_2)$')  # Plot psi(lambda) values with stars
    axes[1, 1].set_title('λ2 and ψ(λ2)')  # Title of the subplot
    axes[1, 1].set_xlabel('Time')  # Label for the x-axis
    axes[1, 1].set_ylabel('Intensity')  # Label for the y-axis
    axes[1, 1].legend()  # Show legend
    axes[1, 1].grid(True)  # Enable grid for better readability

    # Adjust layout to prevent overlapping elements
    plt.tight_layout()

    # Save the figure as a PDF file
    plt.savefig("ar.pdf")

    # Display the figure
    plt.show()

## Homogeneous network

In [ ]:
#Generate the adjecency matrix
def generate_matrix(N, val, prob):
    """
    Parameters:
        N (int): Size of the matrix (N x N).
        val (list): List of possible values that the matrix elements can take - ([0,-1,1] usually)
        prob (list): List of probabilities associated with each value in `val` 

    Returns:
        matrix (ndarray): An NxN adjecency martrix for a given network structure
    """
    # Set a random seed for reproducibility of results
    np.random.seed(123)
    
    # Initialise with zeros
    matrix = np.zeros((N, N))
    
    # Fill the matrix with random values chosen from `val` with corresponding probabilities `prob`
    random_values = np.random.choice(val, size=(N, N), p=prob)
    matrix = random_values 

    return matrix 

In [ ]:
#Redefine poisson_randomness function to take E as an argument
def poisson_randomness(lmbd, timestep, E, psi):
    """
    Parameters:
        lmbd (float): The current intensity parameter (lambda).
        timestep (float): The duration of each time step.
        E (ndarray): Precomputed array of exponential random variables.
        psi (function): Excitation function
    Returns:
        int: The number of events sampled from the Poisson distribution.
    """
    i = 0
    S = 0  
    while S <= psi(lmbd) * timestep:  
        i += 1
        S += E[i]  
    return i -1 

In [ ]:
###EVOLUTION FUNCTIONS FOR NETWORK PROCESS

#Function to calculate the next value of lambda (λ) for a discrete-time Hawkes process on an Erdos-Renyi network.
def lmbd_next_comb(lmbd_prev, G, X_prev, index, timestep=0.1, alpha=0.9, beta=0.5, mu=0.5):
    """
    Parameters:
        lmbd_prev (ndarray): Array of previous lambda values for all nodes in the network.
        G (ndarray): Adjacency matrix representing the connections between nodes in the network.
        X_prev (ndarray): Array of previous event counts for all nodes.
        index (int): The index of the current node for which lambda is being updated.
        timestep (float): Time increment for each step.
        alpha (float): Weight parameter for the influence of the neighbors.
        beta (float): Decay rate of lambda over time.
        mu (float): Baseline intensity for lambda.

    Returns:
        float: The updated lambda value for the specified node.
    """

    lmbd_next = (
        mu * (1 - np.exp(-beta * timestep))  # Baseline intensity contribution
        + np.exp(-beta * timestep) * lmbd_prev[index]  # Decay factor applied to the previous lambda
        + np.exp(-beta * timestep) * alpha / G.shape[0] * np.dot(X_prev, G)  # Influence of neighboring nodes
    )
    return lmbd_next  # Return the updated lambda value for the specified node


#Function to calculate the next value of lambda (λ) for the mean-field process  for a chosen node from the Erdos-Renyi network.
def lmbd_next_mf(prob, psi,lmbd_prev,timestep=0.1, alpha=0.9, beta=0.5, mu=0.5):
    """
    Parameters:
        prob (array): Array of probabilities associated with the ER network
        psi: The excitation function
        lmbd_prev (float): The previous lambda value (intensity).
        timestep (float): Time increment for each step.
        alpha (float): Weight parameter for the influence of the mean field.
        beta (float): Decay rate of lambda over time.
        mu (float): Baseline intensity for lambda.

    Returns:
        float or ndarray: The updated lambda value.
    """
    #Calculating the expected value of theta associate with the ER network
    e = prob[1] - prob[2]
    lmbd_next = np.exp(-beta * timestep) * (lmbd_prev + alpha * timestep * e * psi(lmbd_prev))
    
    return lmbd_next  # Return the updated lambda value

In [ ]:
## SIMULATION FUNCTIONS

#Simulating from the discre-time Hawkes process (DTHP) on an Erdos-Renyi (ER) network
def simulate_comb(prob,psi, E, T, num_nodes, timestep=0.01, alpha=5, beta=10, mu=0.5):
    """
    Parameters:
        prob (list): List of probabilities for generating the adjacency matrix.
        psi: The excitation function
        E (ndarray): Precomputed array of exponential random variables: six values for all nodes and time steps.
        T (int): Total number of time steps for the simulation.
        num_nodes (int): Number of nodes in the network.
        timestep (float): Time increment for each step.
        alpha (float): Weight parameter for the influence of the neighbors.
        beta (float): Decay rate of lambda over time.
        mu (float): Baseline intensity for lambda.
        

    Returns:
        tuple: 
            N (ndarray): Cumulative number of events for each node over time.
            lmbd (ndarray): Intensity (lambda) values for each node over time.
            X (ndarray): Event counts for each node at each time step.
    """
    # Initialize the adjacency matrix for the network using random values with given probabilities
    G = generate_matrix(num_nodes, [0, 1, -1], prob)

    # Initialize matrices )
    N = np.zeros((T, num_nodes))  
    lmbd = np.zeros((T, num_nodes))  
    X = np.zeros((T, num_nodes)) 

    # Set initial values for all nodes
    lmbd[0, :] = np.full(num_nodes, mu)  
    X[0, :] = np.zeros(num_nodes)  

    # Simulate over T time steps
    for i in range(1, T):
        for j in range(num_nodes):
            # Calculate lambda_t
            lmbd[i, j] = lmbd_next_comb(lmbd[i-1, :], G[j, :], X[i-1, :], j, timestep, alpha, beta, mu)
            
            # Sample from Poisson according to lambda_t
            X[i, j] = poisson_randomness(lmbd[i, j], timestep, E[i, j], psi)
            
            # Update the cumulative number of events for each node
            N[i, j] = N[i-1, j] + X[i, j]
    
    return N, lmbd, X 


# SImulating from the mean-field (MF) process
def simulate_mf(prob,psi, E, T, index, timestep=0.01, alpha=5, beta=10, mu=0.5):
    """
    Parameters, as in simulate_comb with the addition of:
        index (int): Index of the node for which to simulate the mean-field approximation.

    Returns:
            N (ndarray): Cumulative number of events over time for the chosen node.
            lmbd (ndarray): Intensity (lambda) values over time for the chosen node.
            X (ndarray): Event counts at each time step for the chosen node.
    """

    # Initialize vectors
    N = np.zeros(T)   
    lmbd = np.zeros(T) 
    X = np.zeros(T)  

   # Set initial values
    X[0] = 0  
    N[0] = X[0] 
    lmbd[0] = mu  

    # Simulate the mean-field approximation over T time steps
    for i in range(1, T):
        # Calculate lambda_t 
        lmbd[i] = lmbd_next_mf(prob, psi,lmbd[i-1],timestep, alpha, beta, mu)
        
        # Sample from Poisson accoring to lmbd_t
        X[i] = poisson_randomness(lmbd[i], timestep, E[i, index])
        
        # Update the cumulative number of events
        N[i] = N[i-1] + X[i]

    return N, lmbd, X  

#Simulating from the DTHP and associated MF used for plotting
def sim_er(chosen_index, num_nodes, T, prob, psi= psi, mu = 0, alpha = 1, beta = 5):
    """
    Parameters:
        chosen_index (int): Index of the node for which to simulate
        num_nodes (int): Number of nodes in the network
        T (int): Number of time-steps
        prob (list): Probablities for the ER network
        mu (float): Baseline intensity
        psi (function): Excitation function


    Returns:
            
    """
    # Genetate six Exponential(1) random variables for each time-step and node combination
    E = np.random.exponential(scale =1, size = (T, num_nodes, 6))

    #Simulate from DTHP
    N_comb, lmbd_comb ,X_comb = simulate_comb(prob,psi, E, T,num_nodes, mu, alpha, beta)

    #Simulate from MF
    N_mf, lmbd_mf, X_mf = simulate_mf(prob,psi, E, T, chosen_index, mu, alpha, beta)

    return lmbd_comb, lmbd_mf

### Different communities plot

In [ ]:

def plot_diffetent_communities(chosen_index, N1, N2, T,prob1, prob2, prob3, prob4):
    """
    Plot the comparison of intensity functions for different community types.

    Parameters:
        chosen_index (int): The index of the node chosen for plotting.
        N1 (int): Number of nodes in the first simulation.
        N2 (int): Number of nodes in the second simulation.
        T (int): Total number of time steps for the simulation.
        prob1 (list): Probabilities for the excitory community.
        prob2 (list): Probabilities for the first mixed community (semi-excitory).
        prob3 (list): Probabilities for the second mixed community (semi-inhibitive).
        prob4 (list): Probabilities for the inhibitive community.
    """

    #Simulate from excitory community
    lmbd_comb, lmbd_mf = sim_er(chosen_index, N1,T, prob1)
    lmbd_comb_more, lmbd = sim_er(chosen_index, N2,T, prob1)

    #Simulate from mixed1 community
    lmbd_comb_mix, lmbd_mf_mix = sim_er(chosen_index, N1,T, prob2)
    lmbd_comb_more_mix, lmbd  = sim_er(chosen_index, N2,T, prob2)

    #Simulate from mixed 2 community
    lmbd_comb_mix2, lmbd_mf_mix2 = sim_er(chosen_index, N1,T, prob3)
    lmbd_comb_more_mix2, lmbd  = sim_er(chosen_index, N2,T, prob3)

    #Simulate from inhibive community
    lmbd_comb_inhibit, lmbd_mf_inhibit = sim_er(chosen_index, N1,T, prob4)
    lmbd_comb_more_inhibit, lmbd  = sim_er(chosen_index, N2,T, prob4)

    plt.figure(figsize=(15, 15))
    bbox_props = dict(boxstyle="round,pad=0.3", edgecolor="black", facecolor="white", alpha=0.7)

    # Plot excitory
    plt.plot(psi(lmbd_mf[:]),'^', markersize = 2, label=r'$\psi(\bar{\lambda}^{E})$', color='red')
    plt.text(20, psi(lmbd_mf[-1]) + 0.02, r'$Excitory$', color='red', fontsize=12, bbox=bbox_props)
    plt.text(len(lmbd_mf), psi(lmbd_mf[-1]), r'$\psi(\bar{\lambda}^{exc})$', color='red', fontsize=12, bbox=bbox_props)
    plt.plot(psi(lmbd_comb[:, chosen_index]), 'o', markersize = 2, label=r'$\psi(\lambda^E_{1000})$', color='blue')
    plt.text(len(lmbd_mf), psi(lmbd_mf[-1]) + 0.02, r'$\psi(\lambda^E_{1000})$', color='blue', fontsize=12, bbox=bbox_props)
    plt.plot(psi(lmbd_comb_more[:, chosen_index]), 'o', markersize = 2, label=r'$\psi(\lambda^E_{10000})$', color='orange')
    plt.text(len(lmbd_mf), psi(lmbd_mf[-1]) - 0.02, r'$\psi(\lambda^E_{10000})$', color='orange', fontsize=12, bbox=bbox_props)

    # Plot mixed1
    plt.plot(psi(lmbd_mf_mix[:]),'^', markersize=2, color='grey')
    plt.text(20, psi(lmbd_mf_mix[-1]) + 0.03, r'$Semi-excitory$', color='grey', fontsize=12, bbox=bbox_props)
    plt.text(len(lmbd_mf), psi(lmbd_mf_mix[-1]), r'$\psi(\bar{\lambda}^{SE})$', color='grey', fontsize=12, bbox=bbox_props)
    plt.plot(psi(lmbd_comb_mix[:, chosen_index]), 'o', markersize=2, color='deepskyblue')
    plt.text(len(lmbd_mf), psi(lmbd_comb_mix[-1, chosen_index]) + 0.04, r'$\psi(\lambda^{SE}_{1000})$', color='deepskyblue', fontsize=12, bbox=bbox_props)
    plt.plot(psi(lmbd_comb_more_mix[:, chosen_index]), 'o', markersize=2, color='goldenrod')
    plt.text(len(lmbd_mf), psi(lmbd_comb_more_mix[-1, chosen_index]) - 0.03, r'$\psi(\lambda^{SE}_{10000})$', color='goldenrod', fontsize=12, bbox=bbox_props)

    # Plot mixed2
    plt.plot(psi(lmbd_mf_mix2[:]),'^', markersize=2, color='sienna')
    plt.text(20, psi(lmbd_mf_mix2[-1]) + 0.03, r'$Semi-inhibitive$', color='sienna', fontsize=12, bbox=bbox_props)
    plt.text(len(lmbd_mf), psi(lmbd_mf_mix2[-1]), r'$\psi(\bar{\lambda}^{SI})$', color='sienna', fontsize=12, bbox=bbox_props)
    plt.plot(psi(lmbd_comb_mix2[:, chosen_index]), 'o', markersize=2, color='darkviolet')
    plt.text(len(lmbd_mf), psi(lmbd_comb_mix2[-1, chosen_index]) + 0.03, r'$\psi(\lambda^{SI}_{1000})$', color='darkviolet', fontsize=12, bbox=bbox_props)
    plt.plot(psi(lmbd_comb_more_mix2[:, chosen_index]), 'o', markersize=2, color='limegreen')
    plt.text(len(lmbd_mf), psi(lmbd_comb_more_mix2[-1, chosen_index]) - 0.015, r'$\psi(\lambda^{SI}_{10000})$', color='limegreen', fontsize=12, bbox=bbox_props)


    # Plot inhibitive
    plt.plot(psi(lmbd_mf_inhibit[:]),'^', markersize=2, color='black')
    plt.text(20, psi(lmbd_mf_inhibit[-1]) - 0.03, r'$Inhibitive$', color='black', fontsize=12, bbox=bbox_props)
    plt.text(len(lmbd_mf), psi(lmbd_mf_inhibit[-1]) - 0.01, r'$\psi(\bar{\lambda}^I)$', color='black', fontsize=12, bbox=bbox_props)
    plt.plot(psi(lmbd_comb_inhibit[:, chosen_index]), 'o', markersize=2, color='purple')
    plt.text(len(lmbd_mf), psi(lmbd_comb_inhibit[-1, chosen_index]) + 0.015, r'$\psi(\lambda^I_{1000})$', color='purple', fontsize=12, bbox=bbox_props)
    plt.plot(psi(lmbd_comb_more_inhibit[:, chosen_index]), 'o', markersize=2, color='green')
    plt.text(len(lmbd_mf), psi(lmbd_comb_more_inhibit[-1, chosen_index]) - 0.02, r'$\psi(\lambda^I_{10000})$', color='green', fontsize=12, bbox=bbox_props)

    # Add labels
    plt.xlabel('Time', fontsize=14)
    plt.ylabel('Intensity', fontsize=14)

    # Add grid
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)

    # Save the plot
    plt.savefig('mf-excitory-inhibitory.pdf')

    # Show the plot
    plt.show()

### Dependence on alpha, beta

### Dependence on N, T plots

In [ ]:
#Calculate the Mean Squared Error (MSE) between the combined process and the mean-field approximation 
# over a range of time steps (T) or a range of node counts (N).

def calculate_mse(chosen_index, prob, N, range):
    """
    Parameters:
        chosen_index (int): Index of the chosen node for comparison.
        prob (list): List of probabilities for the random matrix generation.
        mu (float): Initial value for lambda (rate parameter).
        iterate_over (bool): If True iterate over N, else iterate over T
        range_values (range): Range of values for T or N for the simulation.

    Returns:
        list: A list of MSE values corresponding to each value in range_values.
    """
    # Initialize list to store MSE values
    mse_values = []

    # Iterate over the specified range
    for value in range:
        if N == False:
            T = value
            num_nodes = 500  # Default number of nodes
        elif N == True:
            T = 1000  # Default time step value
            num_nodes = value
        else:
            raise ValueError("iterate_over should be 'T' or 'N'")

        # Simulate the combined process and mean-field approximation
        lmbd_comb, lmbd_mf = sim_er(chosen_index, num_nodes, T, prob)

        # Calculate MSE for the current value of T or N
        mse = np.mean(np.abs(lmbd_comb[:, chosen_index] - lmbd_mf)) / np.max(lmbd_mf)

        # Append the MSE value to the list
        mse_values.append(mse)

    return mse_values


In [ ]:
def plot_NT(chosen_index, prob, N_values, T_values):
    """
    Plot Mean Absolute Deviation (MAD) against the number of nodes (N) and final time (T).

    Parameters:
        chosen_index (int): Index of the chosen node for comparison.
        prob (list): List of probabilities for the random matrix generation.
        N_values (list or array): Range of node counts (N) to iterate over.
        T_values (list or array): Range of time steps (T) to iterate over.

    Returns:
        None: The function saves the plot as a PDF file and displays it.
    """

    #Calculate mse_values
    mse_values = calculate_mse(chosen_index, prob, True, N_values)
    mse_values_T = calculate_mse(chosen_index, prob, False, T_values)

    # Create a figure with two subplots side by side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    # Plot MSE with increasing N
    ax1.plot(N_values, mse_values, marker='o', label='MSE')
    ax1.plot(N_values, mse_values[0] / np.sqrt(N_values/N_values[0]), marker='x', linestyle='--', label='Scaling')
    ax1.set_xlabel('N', fontsize=14)
    ax1.legend(fontsize=12)
    ax1.grid(True)
    ax1.set_title('MAD vs N', fontsize=16)

    # Plot MSE with increasing T
    ax2.plot(T_values[1:], mse_values_T[1:], marker='o')
    ax2.set_xlabel('Final time', fontsize=14)
    ax2.grid(True)
    ax2.set_title('MAD vs T', fontsize=16)

    # Adjust layout to avoid overlapping
    plt.tight_layout()

    # Save and display the figure
    plt.savefig('mse_NT.pdf')
    plt.show()
